In [1]:
import collections
from collections import defaultdict
import sys
import json
import random
from jsmin import jsmin
from io import StringIO
import numpy as np
import copy
import importlib
from functools import partial
import math
import os
import compress_pickle
import itertools

script_n = 'mf_bouton_convergence_210519_random_gen'

sys.path.insert(0, '/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc')
import my_plot
importlib.reload(my_plot)
from my_plot import MyPlotData

import tools_mf_graph
importlib.reload(tools_mf_graph)

fname = ('/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc/' \
         'gen_db/mf_grc/input_graph_210520_all_100_2.gz')
input_graph = compress_pickle.load(fname)

# z_min = 19800-10000
# z_max = 29800+10000
# z_min = 19800
# z_max = 29800
z_min = 19800-5000
z_max = 29800+5000
# GrCs are fully reconstructed and proofread from 90k to 150k
x_min = 320*1000
x_max = 560*1000


# pair_reps = defaultdict(int)
# mfs_within_box = set()
# loc_to_mf = dict()
mf_locs_within_box = set()

for mf_id, mf in input_graph.mfs.items():
    rosette_loc_size = {}
    mf.get_all_mf_locs_size(rosette_loc_size)
    for rosette_loc, size in rosette_loc_size.items():
        x, y, z = rosette_loc
        if x < x_min or x > x_max:
            continue
        if z < z_min or z > z_max:
            continue
        mf_locs_within_box.add(rosette_loc)
#         mfs_within_box.add(mf_id)
#         loc_to_mf[rosette_loc] = mf_id


def get_convergence_pairs(graph):
    convergence_pairs = defaultdict(int)
    for grc_id in graph.grcs:
        grc = graph.grcs[grc_id]
#         connected_edge_count = 0
        connected_locs = []
        for edge in grc.edges:
            mf_id, mf_loc = edge
            if mf_loc not in mf_locs_within_box:
                continue
            connected_locs.append(mf_loc)
        # count
        for pair in itertools.combinations(connected_locs, 2):
            convergence_pairs[tuple(sorted(pair))] += 1
    return convergence_pairs



In [2]:
n_random = 100
rounds = []
for n in range(n_random):
    print('', end='.')
    input_observed = input_graph
    # input_observed = copy.deepcopy(input_graph)
    # input_observed.remove_dendrites(min_len=max_dendrite, max_len=None)
    input_observed.randomize_graph_by_grc2(
#         constant_dendrite_length=22000,
        mf_dist_margin=10000,
        seed=n,
    )
    data = get_convergence_pairs(input_observed)
    rounds.append(data)

....................................................................................................

In [3]:
import compress_pickle
fname = f'{script_n}_dist_{z_min}_{z_max}_{n_random}.gz'
print(fname)
compress_pickle.dump(rounds, fname)

mf_bouton_convergence_210519_random_gen_dist_14800_34800_100.gz
